# Good and bad bash jokes

Compare joke rating with 100 other jokes nearest in time (+/- 50 jokes around). If it is in 25% best it is good, in 25% worst it is bad.

In [53]:
import json
import numpy as np
import pandas as pd

In [54]:
bash_all = json.loads(open('..\\data\\raw\\bash.json', 'r', encoding = 'utf-8').read())

# Add class (good or bad) to each joke
jokes = []
win_len = 100  # Number of jokes to compare
N = len(bash_all)
idx = 4  # index of rating in joke data list
ratings = np.array([b[idx] for b in bash_all])
for i in range(N):
    j1 = max(i - win_len // 2, 0)
    j2 = min(j1 + win_len + 1, N)
    j1 = j2 - win_len - 1
    if bash_all[i][idx] > np.quantile(ratings[j1:j2], 0.75):
        jokes.append(bash_all[i] + [1])
    if bash_all[i][idx] < np.quantile(ratings[j1:j2], 0.25):
        jokes.append(bash_all[i] + [0])

In [55]:
bash = pd.DataFrame(jokes, columns = ['text', 'datetime', 'timestamp', 'id', 'rating', 'is_good'])
bash.head(200)

,text,datetime,timestamp,id,rating,is_good
0,"xxx Посмотри там новость про омича, убившего ж...",2018-10-18 13:44,1539870240,452785,389,0
1,xxx: Добро пожаловать в увлекательный мир Пито...,2018-10-18 13:11,1539868260,452782,180,0
2,Жена (в другой комнате) держит пачку детских ц...,2018-10-18 12:13,1539864780,452779,370,0
3,"Angmarets:\nНичего, бывает. Меня в батлфилде з...",2018-10-18 11:45,1539863100,452778,214,0
4,xxx: Вот чисто ради интереса: как планировали ...,2018-10-18 11:13,1539861180,452776,451,0
...,...,...,...,...,...,...
195,"- Чем в программировании отличаются отклик, от...",2018-09-24 10:44,1537785840,452387,1826,1
196,xxx: говорят все знаки с ограничением 60 км/ч ...,2018-09-24 10:13,1537783980,452386,2934,1
197,На работе прилетели иностранцы. По дороге поте...,2018-09-24 09:45,1537782300,452383,4364,1
198,xxx: считаю необходимым для работы со стрингам...,2018-09-24 09:44,1537782240,452382,433,0


In [56]:
f = open('..\\data\\processed\\bash_jokes.json', 'w', encoding = 'utf-8')
f.write(json.dumps(jokes, ensure_ascii = False))
f.close()

# Good and bad anecdotes

Consider anecdote good if it is in top 3 of the day and its rating is more than 100. Consider bad all anecdotes with rating less than 50.

In [57]:
anecdotes_all = []
for i in range(3):
    f = open('..\\data\\raw\\anekdot_ru{}.json'.format(i+1), 'r', encoding = 'utf-8')
    anecdotes_all += json.loads(f.read())
    f.close()

In [58]:
jokes = []
date_idx = 1  # index of date in anecdote list
rating_idx = 2  # index of date in anecdote list
current_date = anecdotes_all[0][date_idx]
anecdotes_day = []
for a in anecdotes_all:
    date = a[date_idx]
    if date == current_date:
        anecdotes_day.append(a)
    else:
        # Process day anecdotes
        # Top 3 with rating more than 100 is good
        for i in range(min(3, len(anecdotes_day))):
            if anecdotes_day[i][rating_idx] > 100:
                jokes.append(anecdotes_day[i] + [1])
        for i in range(len(anecdotes_day)):
            if anecdotes_day[i][rating_idx] < 50:
                jokes.append(anecdotes_day[i] + [0])
            
        current_date = date
        anecdotes_day = []

In [59]:
an = pd.DataFrame(jokes, columns = ['text', 'date', 'rating', 'votes', 'upvotes', 'downvotes', 'is_good'])
an.head(10)

,text,date,rating,votes,upvotes,downvotes,is_good
0,Из новостей: Сечин попросил у Путина господдер...,2020-05-13,735,869,802,67,1
1,Путин мог наебнуть экономику во имя спасения ж...,2020-05-13,542,998,770,228,1
2,Сегодня в метро видел народ в перчатках: однор...,2020-05-13,491,585,538,47,1
3,"— Здравствуйте, это Лига Супергероев? Запишите...",2020-05-13,49,233,141,92,0
4,"В молодости мной интересовались девушки, бывал...",2020-05-13,43,149,96,53,0
5,"- Здравствуйте, я из РусАДА.\n- Мы все здесь м...",2020-05-13,43,175,109,66,0
6,"Люди, котрые в храмах целуют иссушенные челове...",2020-05-13,32,62,47,15,0
7,"Ну, скажите на милость, разве у настоящего пат...",2020-05-13,28,44,36,8,0
8,"Вот и Песков заразился...Наверное,летал провед...",2020-05-13,25,39,32,7,0
9,Путин сделал сенсационное открытие: когда ребе...,2020-05-13,24,44,34,10,0


In [60]:
# Divede anecdotes into 3 files to keep files less than 50 Mb
N = len(jokes)
for i in range(3):
    f = open('..\\data\\processed\\an_ru{}.json'.format(i), 'w', encoding = 'utf-8')
    f.write(json.dumps(jokes[i*N//3 : (i+1)*N//3], ensure_ascii = False))
    f.close()